In [2]:
import numpy
import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import LeaveOneOut
import math
import numpy.linalg as LA
from sklearn.cluster import KMeans
import random
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection,metrics
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
random.sample
import pandas as pd
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from collections import Counter

import time
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.model_selection import train_test_split,StratifiedKFold
import matplotlib.pyplot as plt
from tflearn.activations import relu
from optparse import OptionParser
from scipy import interp
from tqdm import trange
import tensorflow as tf
%matplotlib notebook

In [3]:
def sample(directory, random_seed):
    all_associations = pd.read_csv(directory + '/all_gpe_pairs.csv')
    known_associations = all_associations.loc[all_associations['label'] == 1]
    print(len(known_associations))
    peco_ids = list(set(known_associations['peco_idx']))
    unknown_associations = all_associations.loc[all_associations['label'] == 0]
    sample_df = known_associations
    for peco_id in peco_ids:
        random_negative = unknown_associations.loc[all_associations['peco_idx'] == peco_id].sample(n=known_associations.loc[all_associations['peco_idx'] == peco_id].shape[0], random_state=random_seed, axis=0, replace=True)
        print(len(random_negative))
        sample_df = pd.concat([sample_df,random_negative], axis=0)

    sample_df.reset_index(drop=True, inplace=True)

    return sample_df

def obtain_data(directory, isbalance):

    if isbalance:
        dtp = sample(directory, random_seed = 1234)
    else:
        dtp = pd.read_csv(directory + '/all_gene_peco_pairs.csv')

    gene_ids = list(set(dtp['gene_idx']))
    peco_ids = list(set(dtp['peco_idx']))
    random.shuffle(gene_ids)
    random.shuffle(peco_ids)
    print('# gene = {} | peco = {}'.format(len(gene_ids), len(peco_ids)))

    gene_test_num = int(len(gene_ids) / 5)
    peco_test_num = int(len(peco_ids) / 5)
    print('# Test: gene = {} | peco = {}'.format(gene_test_num, peco_test_num))    
    
    return dtp, gene_ids, peco_ids, gene_test_num, peco_test_num

def generate_task_Tp_train_test_idx(samples):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1234)

    train_index_all, test_index_all, n = [], [], 0
    train_id_all, test_id_all = [], []
    fold = 0
    for train_idx, test_idx in tqdm(kf.split(samples.iloc[:, 3:])): #train_index与test_index为下标
        print('-------Fold ', fold)
        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)

        train_id_all.append(np.array(dtp.iloc[train_idx][['gene_idx', 'peco_idx']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['gene_idx', 'peco_idx']]))

        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        fold += 1
    return train_index_all, test_index_all, train_id_all, test_id_all

def generate_task_Tg_Tpe_train_test_idx(item, ids, dtp):
    
    test_num = int(len(ids) / 5)
    
    train_index_all, test_index_all = [], []
    train_id_all, test_id_all = [], []
    
    for fold in range(5):
        print('-------Fold ', fold)
        if fold != 4:
            test_ids = ids[fold * test_num : (fold + 1) * test_num]
        else:
            test_ids = ids[fold * test_num :]

        train_ids = list(set(ids) ^ set(test_ids))
        print('# {}: Train = {} | Test = {}'.format(item, len(train_ids), len(test_ids)))

        test_idx = dtp[dtp[item].isin(test_ids)].index.tolist()
        train_idx = dtp[dtp[item].isin(train_ids)].index.tolist()
        random.shuffle(test_idx)
        random.shuffle(train_idx)
        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        assert len(train_idx) + len(test_idx) == len(dtp)

        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)
        
        train_id_all.append(np.array(dtp.iloc[train_idx][['gene_idx', 'peco_idx']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['gene_idx', 'peco_idx']]))
        
    return train_index_all, test_index_all, train_id_all, test_id_all


In [54]:
parser = OptionParser()
parser.add_option("-d", "--d", default=1024, help="The embedding dimension d")
parser.add_option("-n", "--n", default=1, help="global norm to be clipped")
parser.add_option("-k", "--k", default=1, help="The dimension of project matrices k")
parser.add_option("-t", "--t", default="o", help="Test scenario")
(opts, args) = parser.parse_args(args = [])

def check_symmetric(a, tol=1e-8):
    return np.allclose(a, a.T, atol=tol)

def row_normalize(a_matrix, substract_self_loop):
    if substract_self_loop == True:
        np.fill_diagonal(a_matrix,0)
    a_matrix = a_matrix.astype(float)
    row_sums = a_matrix.sum(axis=1)+1e-12
    new_matrix = a_matrix / row_sums[:, np.newaxis]
    new_matrix[np.isnan(new_matrix) | np.isinf(new_matrix)] = 0.0
    return new_matrix

def weight_variable(shape):
    initial = tf.compat.v1.truncated_normal(shape, stddev=0.1,dtype=tf.float32)
    return tf.Variable(initial, dtype=tf.float32)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape,dtype=tf.float32)
    return tf.Variable(initial, dtype=tf.float32)

def a_layer(x,units):
    W = weight_variable([x.get_shape().as_list()[1],units])
    b = bias_variable([units])
    tf.compat.v1.add_to_collection('l2_reg', tf.keras.regularizers.L2(1.0)(W))
    return relu(tf.matmul(x, W) + b)


def bi_layer(x0,x1,sym,dim_pred):
    if sym == False:
        W0p = weight_variable([x0.get_shape().as_list()[1],dim_pred])
        W1p = weight_variable([x1.get_shape().as_list()[1],dim_pred])
        tf.compat.v1.add_to_collection('l2_reg', tf.keras.regularizers.L2(1.0)(W0p))
        tf.compat.v1.add_to_collection('l2_reg', tf.keras.regularizers.L2(1.0)(W1p))
        return tf.matmul(tf.matmul(x0, W0p), 
                            tf.matmul(x1, W1p),transpose_b=True)
    else:
        W0p = weight_variable([x0.get_shape().as_list()[1],dim_pred])
        tf.compat.v1.add_to_collection('l2_reg', tf.keras.regularizers.L2(1.0)(W0p))
        return tf.matmul(tf.matmul(x0, W0p), 
                            tf.matmul(x1, W0p),transpose_b=True)


# load network
network_path = r'../../data/'
save_path = '../result/'
gene_name = pd.read_excel(network_path + 'gene_name.xlsx')
peco_similarity_matrix = np.loadtxt(network_path + 'PSSM.txt')
gene_similarity_matrix = np.loadtxt(network_path + 'GSSM.txt')
gene_peco_associations = np.loadtxt(network_path + 'GPmat.txt',dtype = float)

GPA = gene_peco_associations.T
PS = row_normalize(peco_similarity_matrix, True)
GS = row_normalize(gene_similarity_matrix, True)

[num_peco, num_gene] = GPA.shape
dim_peco = int(opts.d)
dim_gene = int(opts.d)
dim_pred = int(opts.k)
dim_pass = int(opts.d)

In [52]:
class Model(object):
    def __init__(self):
        self._build_model()

    def _build_model(self):
        self.peco_peco = tf.compat.v1.placeholder(tf.float32, [num_peco, num_peco])
        self.peco_peco_normalize = tf.compat.v1.placeholder(tf.float32, [num_peco, num_peco])
        self.gene_gene = tf.compat.v1.placeholder(tf.float32, [num_gene, num_gene])
        self.gene_gene_normalize = tf.compat.v1.placeholder(tf.float32, [num_gene, num_gene])
        self.gene_peco = tf.compat.v1.placeholder(tf.float32, [num_gene, num_peco])
        self.gene_peco_normalize = tf.compat.v1.placeholder(tf.float32, [num_gene, num_peco])
        self.peco_gene = tf.compat.v1.placeholder(tf.float32, [num_peco, num_gene])
        self.peco_gene_normalize = tf.compat.v1.placeholder(tf.float32, [num_peco, num_gene])
        self.peco_gene_mask = tf.compat.v1.placeholder(tf.float32, [num_peco + num_gene, num_peco + num_gene])

        self.A = tf.concat([tf.concat([self.peco_peco, self.peco_gene], 1),
                            tf.concat([self.gene_peco, self.gene_gene], 1)], 0)

        self.peco_embedding = weight_variable([num_peco, dim_peco])
        self.gene_embedding = weight_variable([num_gene, dim_gene])
        tf.compat.v1.add_to_collection('l2_reg', tf.keras.regularizers.L2(1.0)(self.peco_embedding))
        tf.compat.v1.add_to_collection('l2_reg', tf.keras.regularizers.L2(1.0)(self.gene_embedding))

        W0 = weight_variable([dim_pass + dim_peco, dim_peco])
        b0 = bias_variable([dim_peco])
        tf.compat.v1.add_to_collection('l2_reg', tf.keras.regularizers.L2(1.0)(W0))

        peco_vector0 = tf.compat.v1.nn.l2_normalize(relu(tf.matmul(
            tf.concat([tf.matmul(self.peco_gene_normalize,
                                 a_layer(self.gene_embedding, dim_pass)) +
                       tf.matmul(self.peco_peco_normalize,
                                 a_layer(self.peco_embedding, dim_pass)),
                       self.peco_embedding], axis=1), W0) + b0), dim=1)

        gene_vector0 = tf.compat.v1.nn.l2_normalize(relu(tf.matmul(
            tf.concat([tf.matmul(self.gene_gene_normalize,
                                 a_layer(self.gene_embedding, dim_pass)) +
                       tf.matmul(self.gene_peco_normalize,
                                 a_layer(self.peco_embedding, dim_pass)),
                       self.gene_embedding], axis=1), W0) + b0), dim=1)

        self.peco_representation = peco_vector0
        self.gene_representation = gene_vector0
        self.features_matrix = tf.concat([self.peco_representation, self.gene_representation], 0)
        self.A_reconstruct = bi_layer(self.features_matrix, self.features_matrix, sym=True, dim_pred=dim_pred)

        tmp = tf.multiply(self.peco_gene_mask, (self.A_reconstruct - self.A))
        self.A_reconstruct_loss = tf.reduce_sum(tf.multiply(tmp, tmp))
        self.l2_loss = tf.add_n(tf.compat.v1.get_collection("l2_reg"))
        self.loss = self.A_reconstruct_loss + self.l2_loss

In [12]:
graph = tf.compat.v1.get_default_graph()
with graph.as_default():
    model = Model()
    learning_rate = tf.compat.v1.placeholder(tf.float32, [])
    total_loss = model.loss

    optimize = tf.compat.v1.train.AdamOptimizer(learning_rate)
    gradients, variables = zip(*optimize.compute_gradients(total_loss))
    gradients, _ = tf.clip_by_global_norm(gradients, int(opts.n))
    optimizer = optimize.apply_gradients(zip(gradients, variables))

    DR_pred = model.A_reconstruct[:num_peco, num_peco:]
    RD_pred = model.A_reconstruct[num_peco:, :num_peco]
    eval_pred = (DR_pred + tf.transpose(RD_pred, perm=[1,0])) / 2.0


def divide_known_unknown_associations(A, exception=None, special=None):
    known = []
    unknown = []
    if special != None:
        for j in range(A.shape[1]):
            if A[special][j] == 1:
                known.append([special,j,1])
            else:
                unknown.append([special,j,0])
    else:
        for i in range(A.shape[0]):
            if i == exception: pass
            for j in range(A.shape[1]):
                if A[i][j] == 1:
                    known.append([i,j,1])
                else:
                    unknown.append([i,j,0])

    return np.array(known), np.array(unknown)


In [13]:
def performances(y_true, y_pred, y_prob):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()

    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if tp+fn == 0:
        recall = 0
    else:
        recall = tp / (tp+fn)
    
    if tp+fp == 0:
        precision = 0
    else:
        precision = tp / (tp+fp)
    
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2*precision*recall / (precision+recall)
    
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}'.format(accuracy, precision, recall, f1, roc_auc, aupr))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr)

def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred.reshape(-1)]

In [14]:
def train_and_evaluate(GPAtrain, GPAtest, graph, verbose=True, num_steps = 2000):
    mask = np.zeros((num_peco,num_gene))
    peco_gene = np.zeros((num_peco,num_gene))
    print(num_peco)
    print(num_gene)
    for ele in GPAtrain:
        peco_gene[ele[0],ele[1]] = ele[2]
        mask[ele[0],ele[1]] = 1
    gene_peco = peco_gene.T
    peco_gene_normalize = row_normalize(peco_gene,False)
    gene_peco_normalize = row_normalize(gene_peco,False)
    mask = np.concatenate([np.concatenate([np.ones((num_peco, num_peco)), mask], axis=1),
                           np.concatenate([mask.T, np.ones((num_gene, num_gene))], axis=1)], axis=0)


    lr = 0.005
    scores = []
    labels = []
    min_loss = float('inf')
    with tf.compat.v1.Session(graph=graph) as sess:
        tf.compat.v1.global_variables_initializer().run()
        for i in trange(num_steps):
            _, tloss, results = sess.run([optimizer,total_loss,eval_pred],
                                        feed_dict={model.peco_gene:peco_gene, model.peco_gene_normalize:peco_gene_normalize,
                                        model.gene_peco:gene_peco, model.gene_peco_normalize:gene_peco_normalize,
                                        model.peco_peco:peco_similarity_matrix, model.gene_gene:gene_similarity_matrix,
                                        model.peco_peco_normalize:PS, model.gene_gene_normalize:GS,
                                        model.peco_gene_mask:mask, learning_rate: lr})
            #every 20 steps of gradient descent, evaluate the performance, other choices of this number are possible
            if i % 20 == 0 and verbose == True:
#                 print('step',i,'total loss',tloss)

                if tloss <= min_loss:
                    min_loss = tloss
                    best_results = results


        for ele in GPAtest:
            scores.append(best_results[ele[0], ele[1]])
            labels.append(ele[2])
        

    return scores, labels

def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred]

# Tp, balance

In [28]:
sample(r'../../data/', random_seed = 1234)
directory = r'../../data'
dtp, gene_ids, peco_ids, gene_test_num, peco_test_num = obtain_data(directory, isbalance = True)
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(dtp)

5it [00:00, 119.12it/s]

# gene = 11177 | peco = 24
# Test: gene = 2235 | peco = 4
-------Fold  0
# Pairs: Train = 37692 | Test = 9424
-------Fold  1
# Pairs: Train = 37693 | Test = 9423
-------Fold  2
# Pairs: Train = 37693 | Test = 9423
-------Fold  3
# Pairs: Train = 37693 | Test = 9423
-------Fold  4
# Pairs: Train = 37693 | Test = 9423


In [ ]:
epochs = 2000

tpr_fold = []
precision_fold = []
for train_idx, test_idx in zip(train_index_all, test_index_all):
    GPAtrain = np.array(dtp.iloc[train_idx])
    GPAtest = np.array(dtp.iloc[test_idx])

    scores, labels = train_and_evaluate(GPAtrain=GPAtrain, GPAtest=GPAtest,
                                        graph=graph, num_steps=epochs)
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

# Tg, balance

In [29]:
directory = r'../../data'
dtp, gene_ids, peco_ids, gene_test_num, peco_test_num = obtain_data(directory, isbalance = True)
item = 'gene_idx'
ids = gene_ids
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tg_Tpe_train_test_idx(item, ids, dtp)

epochs = 2000

tpr_fold = []
precision_fold = []
mean_fpr = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
for train_idx, test_idx in zip(train_index_all, test_index_all):
    GPAtrain = np.array(dtp.iloc[train_idx])
    GPAtest = np.array(dtp.iloc[test_idx])
    scores, labels = train_and_evaluate(GPAtrain=GPAtrain, GPAtest=GPAtest,
                                        graph=graph, num_steps=epochs)
    
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

# gene = 11177 | peco = 24
# Test: gene = 2235 | peco = 4
-------Fold  0
# gene_idx: Train = 8942 | Test = 2235
# Pairs: Train = 37625 | Test = 9491
-------Fold  1
# gene_idx: Train = 8942 | Test = 2235
# Pairs: Train = 37721 | Test = 9395
-------Fold  2
# gene_idx: Train = 8942 | Test = 2235
# Pairs: Train = 37708 | Test = 9408
-------Fold  3
# gene_idx: Train = 8942 | Test = 2235
# Pairs: Train = 37619 | Test = 9497
-------Fold  4
# gene_idx: Train = 8940 | Test = 2237
# Pairs: Train = 37791 | Test = 9325
32
12187


100%|████████████████████████████████████████████████████████████████████████████| 2000/2000 [8:00:53<00:00, 14.43s/it]
<ipython-input-26-bff51fc9c44e>:47: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred]


tn = 4318, fp = 427, fn = 1639, tp = 3107
y_pred: 0 = 5957 | 1 = 3534
y_true: 0 = 4745 | 1 = 4746
acc=0.7823|precision=0.8792|recall=0.6547|f1=0.7505|auc=0.8773|aupr=0.8901
32
12187


100%|████████████████████████████████████████████████████████████████████████████| 2000/2000 [7:58:31<00:00, 14.36s/it]
<ipython-input-26-bff51fc9c44e>:47: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred]


tn = 4184, fp = 470, fn = 1474, tp = 3267
y_pred: 0 = 5658 | 1 = 3737
y_true: 0 = 4654 | 1 = 4741
acc=0.7931|precision=0.8742|recall=0.6891|f1=0.7707|auc=0.8969|aupr=0.9053
32
12187


...

100%|████████████████████████████████████████████████████████████████████████████| 2000/2000 [6:57:00<00:00, 12.51s/it]
<ipython-input-26-bff51fc9c44e>:47: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred]


tn = 4188, fp = 596, fn = 1536, tp = 3177
y_pred: 0 = 5724 | 1 = 3773
y_true: 0 = 4784 | 1 = 4713
acc=0.7755|precision=0.8420|recall=0.6741|f1=0.7488|auc=0.8539|aupr=0.8745
32
12187


100%|████████████████████████████████████████████████████████████████████████████| 2000/2000 [6:44:08<00:00, 12.12s/it]


tn = 4193, fp = 494, fn = 1596, tp = 3042
y_pred: 0 = 5789 | 1 = 3536
y_true: 0 = 4687 | 1 = 4638
acc=0.7759|precision=0.8603|recall=0.6559|f1=0.7443|auc=0.8618|aupr=0.8721


<ipython-input-26-bff51fc9c44e>:47: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred]


# Tpe, balance

In [56]:
directory = r'../../data'
dtp, gene_ids, peco_ids, gene_test_num, peco_test_num = obtain_data(directory, isbalance = True)
item = 'peco_idx'
ids = peco_ids
train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tg_Tpe_train_test_idx(item, ids, dtp)

epochs = 30

for train_idx, test_idx in zip(train_index_all, test_index_all):
    GPAtrain = np.array(dtp.iloc[train_idx])
    GPAtest = np.array(dtp.iloc[test_idx])
    scores, labels = train_and_evaluate(GPAtrain=GPAtrain, GPAtest=GPAtest,
                                        graph=graph, num_steps=epochs)
    
    y_test_prob = scores
    y_test_pred = transfer(y_test_prob)
    performances_test = performances(labels, y_test_pred, y_test_prob)

23558
1528
10396
86
121
1179
126
5
14
45
4992
7
67
312
853
148
29
831
45
476
2239
1
51
1
6
# gene = 11177 | peco = 24
# Test: gene = 2235 | peco = 4
-------Fold  0
# peco_idx: Train = 20 | Test = 4
# Pairs: Train = 40644 | Test = 6472
-------Fold  1
# peco_idx: Train = 20 | Test = 4
# Pairs: Train = 46394 | Test = 722
-------Fold  2
# peco_idx: Train = 20 | Test = 4
# Pairs: Train = 44006 | Test = 3110
-------Fold  3
# peco_idx: Train = 20 | Test = 4
# Pairs: Train = 43714 | Test = 3402
-------Fold  4
# peco_idx: Train = 16 | Test = 8
# Pairs: Train = 13706 | Test = 33410
32
12187


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [06:00<00:00, 12.01s/it]
<ipython-input-14-bff51fc9c44e>:47: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred]


tn = 45, fp = 3191, fn = 49, tp = 3187
y_pred: 0 = 94 | 1 = 6378
y_true: 0 = 3236 | 1 = 3236
acc=0.4994|precision=0.4997|recall=0.9849|f1=0.6630|auc=0.5192|aupr=0.5418
32
12187


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [05:43<00:00, 11.44s/it]
<ipython-input-14-bff51fc9c44e>:47: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred]


tn = 0, fp = 361, fn = 0, tp = 361
y_pred: 0 = 0 | 1 = 722
y_true: 0 = 361 | 1 = 361
acc=0.5000|precision=0.5000|recall=1.0000|f1=0.6667|auc=0.6736|aupr=0.6157
32
12187


...

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [05:53<00:00, 11.79s/it]
<ipython-input-14-bff51fc9c44e>:47: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred]


tn = 1698, fp = 3, fn = 1700, tp = 1
y_pred: 0 = 3398 | 1 = 4
y_true: 0 = 1701 | 1 = 1701
acc=0.4994|precision=0.2500|recall=0.0006|f1=0.0012|auc=0.5890|aupr=0.5763
32
12187


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [05:48<00:00, 11.60s/it]


tn = 11179, fp = 5526, fn = 11096, tp = 5609
y_pred: 0 = 22275 | 1 = 11135
y_true: 0 = 16705 | 1 = 16705
acc=0.5025|precision=0.5037|recall=0.3358|f1=0.4029|auc=0.4887|aupr=0.4902


<ipython-input-14-bff51fc9c44e>:47: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return [[0,1][x>0.5] for x in y_pred]
